In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume-dataset/Resume/Resume.csv
/kaggle/input/resume-dataset/data/data/DESIGNER/22506245.pdf
/kaggle/input/resume-dataset/data/data/BPO/95625660.pdf
/kaggle/input/resume-dataset/data/data/FINANCE/31948488.pdf
/kaggle/input/resume-dataset/data/data/CONSTRUCTION/12890045.pdf
/kaggle/input/resume-dataset/data/data/SALES/16121015.pdf
/kaggle/input/resume-dataset/data/data/AUTOMOBILE/99680385.pdf
/kaggle/input/resume-dataset/data/data/CONSULTANT/12955994.pdf
/kaggle/input/resume-dataset/data/data/CHEF/19831366.pdf
/kaggle/input/resume-dataset/data/data/APPAREL/29028935.pdf
/kaggle/input/resume-dataset/data/data/AGRICULTURE/12341902.pdf
/kaggle/input/resume-dataset/data/data/TEACHER/21773106.pdf
/kaggle/input/resume-dataset/data/data/HR/26202430.pdf
/kaggle/input/resume-dataset/data/data/DIGITAL-MEDIA/24658786.pdf
/kaggle/input/resume-dataset/data/data/ACCOUNTANT/27558837.pdf
/kaggle/input/resume-dataset/data/data/HEALTHCARE/18365443.pdf
/kaggle/input/resume-dataset/data/data/

In [11]:
resume_df = pd.read_csv("/kaggle/input/resume-dataset/Resume/Resume.csv")

In [12]:
resume_df.head(5)

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [13]:
resume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [14]:
resume_df["all_text"] = resume_df["Resume_str"] + " " + resume_df["Resume_html"]

In [15]:
len(resume_df["Category"].unique())

24

In [16]:
resume_df.shape

(2484, 5)

In [17]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

df = resume_df

# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Load the pre-trained BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = len(df['Category'].unique())
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Define batch size and number of epochs
batch_size = 20
num_epochs = 3

# Train the BERT model in batches
for epoch in range(num_epochs):
    for batch_start in range(0, len(train_df), batch_size):
        batch_end = min(batch_start + batch_size, len(train_df))
        batch_df = train_df.iloc[batch_start:batch_end]
        
        batch_texts = batch_df['Resume_str'].tolist()
        batch_labels = batch_df['Category'].tolist()
        
        # Tokenize the text data and features
        batch_inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        
        # Convert labels to numerical format
        label_mapping = {label: index for index, label in enumerate(df['Category'].unique())}
        batch_labels_tensor = torch.tensor([label_mapping[label] for label in batch_labels], dtype=torch.long)
        
        # Train the model
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
        loss_fn = torch.nn.CrossEntropyLoss()
        
        model.train()
        optimizer.zero_grad()
        outputs = model(**batch_inputs, labels=batch_labels_tensor)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(">", end=" ")
    print(f"Epoch {epoch+1}, Batch {batch_start//batch_size + 1}, Loss: {loss.item()}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > Epoch 1, Batch 100, Loss: 3.186096668243408
> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > Epoch 2, Batch 100, Loss: 2.709461212158203
> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > Epoch 3, Batch 100, Loss: 2.1091134548187256


In [18]:
from transformers import BertModel, BertTokenizer

# Save the model and tokenizer
output_dir = "bert_saved_model_1"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('bert_saved_model_1/tokenizer_config.json',
 'bert_saved_model_1/special_tokens_map.json',
 'bert_saved_model_1/vocab.txt',
 'bert_saved_model_1/added_tokens.json')

In [19]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score
from tqdm import tqdm  # For progress bar



# Define batch size
batch_size = 8

# Tokenize and evaluate the test dataset in batches
all_predicted_labels = []
all_true_labels = []

for batch_start in tqdm(range(0, len(test_df), batch_size)):
    batch_end = min(batch_start + batch_size, len(test_df))
    batch_df = test_df.iloc[batch_start:batch_end]
    
    batch_texts = batch_df['Resume_str'].tolist()
    batch_labels = batch_df['Category'].tolist()
    
    # Tokenize the text data
    batch_inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    
    # Convert labels to numerical format
    label_mapping = {label: index for index, label in enumerate(test_df['Category'].unique())}
    batch_labels_tensor = torch.tensor([label_mapping[label] for label in batch_labels], dtype=torch.long)
    
    model.eval()
    with torch.no_grad():
        batch_outputs = model(**batch_inputs)
        batch_predicted_labels = torch.argmax(batch_outputs.logits, dim=1)
    
    all_predicted_labels.extend(batch_predicted_labels.cpu().numpy())
    all_true_labels.extend(batch_labels_tensor.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_true_labels, all_predicted_labels)
print(f"Accuracy on test dataset: {accuracy:.2f}")


100%|██████████| 63/63 [07:47<00:00,  7.43s/it]

Accuracy on test dataset: 0.06


In [20]:
test_df.shape

(497, 5)

In [21]:
import torch
from sklearn.metrics import accuracy_score, classification_report

classification_rep = classification_report(all_true_labels, all_predicted_labels, target_names=df['Category'].unique())

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)


Accuracy: 0.06237424547283702
Classification Report:
                        precision    recall  f1-score   support

                    HR       0.00      0.00      0.00        22
              DESIGNER       0.00      0.00      0.00        25
INFORMATION-TECHNOLOGY       0.00      0.00      0.00        34
               TEACHER       0.00      0.00      0.00        24
              ADVOCATE       0.00      0.00      0.00        23
  BUSINESS-DEVELOPMENT       0.90      1.00      0.95        27
            HEALTHCARE       0.00      0.00      0.00        19
               FITNESS       0.00      0.00      0.00        20
           AGRICULTURE       0.00      0.00      0.00        29
                   BPO       0.00      0.00      0.00        18
                 SALES       0.03      0.17      0.05         6
            CONSULTANT       0.07      0.03      0.05        30
         DIGITAL-MEDIA       0.00      0.00      0.00        18
            AUTOMOBILE       0.00      0.00      0

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# MultinomialNB model

In [22]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score



# Convert data to DataFrame
df = resume_df

# Split the data
train_df, test_df = train_test_split(df, test_size=0.01, random_state=42)

# Preprocessing and feature extraction
vectorizer = CountVectorizer(stop_words="english")
X_train = vectorizer.fit_transform(train_df["Resume_str"])
X_test = vectorizer.transform(test_df["Resume_str"])

# Train a model
model = MultinomialNB()
model.fit(X_train, train_df["Category"])

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(test_df["Category"], predictions)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.56


In [23]:
import torch
from sklearn.metrics import accuracy_score, classification_report

classification_rep = classification_report(test_df["Category"], predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)


Accuracy: 0.56
Classification Report:
                        precision    recall  f1-score   support

              ADVOCATE       0.20      1.00      0.33         1
                  ARTS       0.00      0.00      0.00         2
            AUTOMOBILE       0.00      0.00      0.00         1
               BANKING       1.00      0.33      0.50         3
  BUSINESS-DEVELOPMENT       0.50      0.50      0.50         2
                  CHEF       1.00      1.00      1.00         1
          CONSTRUCTION       0.50      1.00      0.67         1
              DESIGNER       1.00      1.00      1.00         4
         DIGITAL-MEDIA       1.00      0.50      0.67         2
           ENGINEERING       0.00      0.00      0.00         0
            HEALTHCARE       0.00      0.00      0.00         2
                    HR       0.67      1.00      0.80         2
INFORMATION-TECHNOLOGY       0.00      0.00      0.00         0
      PUBLIC-RELATIONS       0.00      0.00      0.00         0
 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.

**End this endpoint, we well select MultinomialNB model is best then BERT for given dataset.**

In [24]:
#so we will chosen MultinomialNB model
# Save the model to a file

import joblib


model_filename = 'final model.pkl'
joblib.dump(model, model_filename)


predictions


array(['ADVOCATE', 'PUBLIC-RELATIONS', 'CONSTRUCTION', 'CHEF', 'BANKING',
       'HEALTHCARE', 'DESIGNER', 'BUSINESS-DEVELOPMENT', 'TEACHER',
       'DESIGNER', 'CONSTRUCTION', 'ADVOCATE', 'SALES',
       'INFORMATION-TECHNOLOGY', 'ADVOCATE', 'ADVOCATE', 'ENGINEERING',
       'ADVOCATE', 'HR', 'HR', 'DESIGNER', 'HR', 'DESIGNER',
       'BUSINESS-DEVELOPMENT', 'DIGITAL-MEDIA'], dtype='<U22')

In [25]:
# Load the saved model from a file
loaded_model = joblib.load(model_filename)


In [26]:
submission_df = test_df.iloc[:, [0,3]]

In [27]:
submission_df

,ID,Category
420,99244405,TEACHER
1309,17562754,DIGITAL-MEDIA
2023,30311725,CONSTRUCTION
1360,19007667,CHEF
2186,11065180,BANKING
602,39237915,BUSINESS-DEVELOPMENT
173,17199951,DESIGNER
599,18236085,BUSINESS-DEVELOPMENT
435,79663360,TEACHER
188,62312955,DESIGNER


In [28]:
submission_df["Category"] = predictions

/tmp/ipykernel_32/1813375056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df["Category"] = predictions


In [29]:
submission_df

,ID,Category
420,99244405,ADVOCATE
1309,17562754,PUBLIC-RELATIONS
2023,30311725,CONSTRUCTION
1360,19007667,CHEF
2186,11065180,BANKING
602,39237915,HEALTHCARE
173,17199951,DESIGNER
599,18236085,BUSINESS-DEVELOPMENT
435,79663360,TEACHER
188,62312955,DESIGNER


In [30]:
submission_df.to_csv("categorized_resumes.csv")